# Grokking Simplicity

## Chapter 1, 2 and 3

### 1.1. Defining Functional Programming

We start with a definition of **Function Programming (FP)**. It is a programming paradigm characterized by
the use of **pure functions** and the avoidance of **side effects**.

What are **pure functions**? In programming, a pure function is a function that depends *only* on it's arguments,
such that it always returns the same value if the same same arguments are passed.

What are **side effects**? A side effect is anything that a function does that affects the "outside"/"global scope". For example,
consider the following function:

In [24]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Main/EMAp/Julia_Tutorials/FunctionalProgramming`


In [25]:
function squareprint(x::Real)
    print("squaring")
    x^2
end
squareprint(2)

squaring

4

This function has the side-effect of printing "squaring". Another possible side-effect would be sending an email, writing a log, and so on. 

From this definition, it might looks like FP is useless in the real world, since we need side-effects for most
practical programs. The answer is that, *actually*, FP allows side-effects, but in a very controlled manner. Hence one of it's utility.
By controlling side-effects (when an how they occur), we can better understand when some kind of error is going on.

### 1.2 Data, Calculation and Action 

In FP, every chunk of code can be classified in Data, Calculation or Action.
Actions are usually pieces of code that result in side-effects, and thus, it matters **when** and **how
many times** such piece of code is used. For example, a function `sendemail()` would be an Action,
since calling this function twice would result in sending two emails. Yet, a function
such as `sum(x::Vector{Real})` would be a calculation, since calling it several times does
not alter anything, it just calculates the same sum over and over again.

Hence, Actions are the most "dangerous" pieces of code. On the other hand,
Data is "inert", while Calculations can be "executed". Thus, there is sort of a hierarchy of
complixity to simplicity, where Actions are the most complex and Data are the most simple.

The advantages of **Data** is that it can be more easily turned into bits, and then transfered to another machine
or program. Also, it's easier to assert equality, compared to calculations and actions.

Another important aspect of data is that it can be interpreted in many ways.

#### CONTEXT MATTERS

Consider "Reading the records in a DataBase". Is this a Calculation or Action? One might think that it's
like a calculation at first, since if I read the database many times, it would always return the records.
Yet, this can be better understood as an Action. Why? Because of the context. Consider here that the
argument for the function `readdb(x::DataBase)` is a database, and that, for the same database,
we can get different results, depending on what is stored in the database.

Consider now the function "Return the odd values".
Depending on the argument we are considering, this can be a Calculation or an Action. This difference
we'll usually happen if the arguments that a function receives are mutable or immutable.
Note that a database is mutable, so the function will return different values depending on
when it's called. If we made the database immutable, then we could actually consider the `readdb` function
as a calculation, since it would always return the same value whenever it was called with an specific database.

In Julia, since `::Array` is a mutable datatype, this means that a function such as `return_odd_numbers(A::Array)` 
would be an action, while `return_odd_number(A::Int)` would be a calculation.

## 2. Sending Coupom to Email 

Let's code the example of sending the emails with coupons from **Chapter 3** of the book.

We simplify the example by using dictionaries as databases.

In [39]:
using StructArrays

In [32]:
struct Subscriber
    email::String
    rec_count::Int
    Subscriber(email, rec_count) = rec_count ≥ 0 ? new(email, rec_count) : error("rec_count must be ≥ 0.")
end

struct Coupon
    code::String
    rank::String
    Coupon(code, rank) = rank in ["good", "bad", "best"] ? new(code, rank) : error("rank must be 'good', 'bad' or 'best'.")
end

In [64]:
Subscriber("Davi",10)
Coupon("MYCOUPON","good")

Coupon("MYCOUPON", "good")

In [65]:
emails = ["john@coldmail.com", "sam@pmail.co", "linda1989@oal.com", "jan1940@ahoy.com", "mrbig@pmail.co", "lol@lol.lol",]
rec_count = [2, 16, 1, 0, 25, 0]

EMAILS = StructArray{Subscriber}(email = emails, rec_count = rec_count)

6-element StructArray(::Vector{String}, ::Vector{Int64}) with eltype Subscriber:
 Subscriber("john@coldmail.com", 2)
 Subscriber("sam@pmail.co", 16)
 Subscriber("linda1989@oal.com", 1)
 Subscriber("jan1940@ahoy.com", 0)
 Subscriber("mrbig@pmail.co", 25)
 Subscriber("lol@lol.lol", 0)

In [66]:
coupons = ["MAYDISCOUNT", "10PERCENT", "PROMOTION45", "IHEARTYOU", "GETADEAL", "ILIKEDISCOUNTS",]
ranks   = ["good", "bad", "best", "bad", "best", "good",]

COUPONS = StructArray{Coupon}(code = coupons, rank = ranks)

6-element StructArray(::Vector{String}, ::Vector{String}) with eltype Coupon:
 Coupon("MAYDISCOUNT", "good")
 Coupon("10PERCENT", "bad")
 Coupon("PROMOTION45", "best")
 Coupon("IHEARTYOU", "bad")
 Coupon("GETADEAL", "best")
 Coupon("ILIKEDISCOUNTS", "good")

In [67]:
map(EMAILS) do x getfield(x, :email) end

6-element Vector{String}:
 "john@coldmail.com"
 "sam@pmail.co"
 "linda1989@oal.com"
 "jan1940@ahoy.com"
 "mrbig@pmail.co"
 "lol@lol.lol"

In [76]:
EMAILS.email

6-element Vector{String}:
 "john@coldmail.com"
 "sam@pmail.co"
 "linda1989@oal.com"
 "jan1940@ahoy.com"
 "mrbig@pmail.co"
 "lol@lol.lol"

In [81]:
function subcuponrank(subscriber::Subscriber)
    subscriber.rec_count ≥ 10 ? "best" : "good"
end

map(subcuponrank, EMAILS)

6-element Vector{String}:
 "good"
 "best"
 "good"
 "good"
 "best"
 "good"

In [93]:
flatmap(f,x) = collect(Iterators.flatten(map(f,x)));

In [96]:
COUPONS

6-element StructArray(::Vector{String}, ::Vector{String}) with eltype Coupon:
 Coupon("MAYDISCOUNT", "good")
 Coupon("10PERCENT", "bad")
 Coupon("PROMOTION45", "best")
 Coupon("IHEARTYOU", "bad")
 Coupon("GETADEAL", "best")
 Coupon("ILIKEDISCOUNTS", "good")

In [105]:
function selectcuponsbyrank(coupons, rank)
    flatmap(coupons) do x
        x.rank == rank ? x.code : ()
    end
end

selectcuponsbyrank (generic function with 1 method)